In [ ]:
import numpy as np
import matplotlib
import pandas as pd
import tensorflow as tf

dataset = pd.read_csv('/Users/wzhang/Downloads/lake_erie_habs_W4_6_8_9_12_13_16_2013-2020.xlsx - HAB_data (1).csv')

train = dataset.iloc[:743,:]
test = dataset.iloc[743:,:]
print(train)
print(test)
X_train = pd.DataFrame(train.iloc[:,4:15].values)
Y_train = train.iloc[:,16:].values
print(X_train.shape)
print(Y_train.shape)
X_test = pd.DataFrame(test.iloc[:,4:15].values)
Y_test = test.iloc[:,16:].values
print(X_test.shape)
print(Y_test.shape)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [ ]:
from keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization, GRU, LayerNormalization, MultiHeadAttention,TimeDistributed, Input, Flatten
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def create_GRU():
    model = Sequential()

    model.add(GRU(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))

    model.add(GRU(units=100, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(GRU(units=100))
    model.add(Dropout(0.2))

    model.add(Dense(units=1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0008), loss='mae')
    model.fit(
        X_train, Y_train, validation_split = 0.2, batch_size=24, epochs=2000
    )
    return model

In [ ]:
from sklearn.linear_model import Lasso
def create_lasso():
    model = Lasso(alpha=100)
    model.fit(X_train,Y_train)
    return model


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
def create_LSTM():

    # Define the LSTM model
    model = Sequential()

    model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units=100, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(units=100))
    model.add(Dropout(0.2))

    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')


    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00075), loss='mae')
    # Fit the model to the training data
    history = model.fit(X_train, Y_train, epochs=1000, batch_size=64, validation_data=(X_test, Y_test), shuffle=False)
    return model


In [ ]:
def create_mlp():
    # Initialize MLP model
    mlp_model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1)  # Output layer for regression task
    ])

    # Compile the model
    mlp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001), loss='mae')

    # Print model summary
    mlp_model.summary()

    # Train the model
    history = mlp_model.fit(X_train, Y_train, epochs=300, batch_size=32, validation_split=0.2)

    # Evaluate model performance
    return mlp_model


In [ ]:
def create_rf():
    # Initialize Random Forest regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    rf_model.fit(X_train, Y_train)
    return rf_model




In [ ]:
from sklearn.linear_model import Ridge
def create_ridge():
    model = Ridge(alpha=100)
    model.fit(X_train,Y_train)
    return model

In [ ]:
from sklearn.svm import SVR
def create_SVM():
    # Initialize SVM regressor
    svm_model = SVR(kernel='rbf', C=10000, gamma='scale')

    # Train the model
    svm_model.fit(X_train, Y_train)
    return svm_model


In [ ]:
import lightgbm as lgb
def create_lightgbm():
    params = { 
    'objective': 'regression', 
    'metric': 'rmse', 
    'boosting_type': 'gbdt', 
    'num_leaves': 31, 
    'learning_rate': 0.05, 
    'feature_fraction': 0.9, 
    }
    train_data = lgb.Dataset(X_train,label=Y_train)
    num_round = 100
    bst = lgb.train(params, train_data, num_round) 
    from sklearn.metrics import mean_squared_error as mse 
    from lightgbm import LGBMRegressor 
    
    # Create an instance of the LightGBM Regressor with the RMSE metric. 
    model = LGBMRegressor(metric='rmse') 
    
    # Train the model using the training data. 
    model.fit(X_train, Y_train) 

    res = model.predict(X_test) 
    return model


In [ ]:
# gru = create_GRU()
# lasso = create_lasso()
# lstm = create_LSTM()
mlp = create_mlp()
rf = create_rf()
lightgbm = create_lightgbm()
svm = create_SVM()

# ridge = create_ridge()
# svm = create_SVM()

In [ ]:
import deepforest
from deepforest import CascadeForestRegressor

In [ ]:
from tensorflow.keras.layers import concatenate


# gruPredict = gru.predict(X_train)
# lassoPredict = lasso.predict(X_train)
# lstmPredict = lstm.predict(X_train)
mlpPredict = mlp.predict(X_train)
lightgbmPredict = lightgbm.predict(X_train)
# ridgePredict = ridge.predict(X_train)
# svmPredict = svm.predict(X_train)
# rfPredict = rf.predict(X_train)
# lassoPredict.resize(671,1)
mlpPredict.resize(743,1)
lightgbmPredict.resize(743,1)
# svmPredict.resize(671,1)
trainData = concatenate([mlpPredict,lightgbmPredict])

In [ ]:
model = CascadeForestRegressor(n_estimators = 50, criterion='squared_error', backend='sklearn')

In [ ]:
import numpy as np

In [ ]:
np.__version__

In [ ]:
model.fit(trainData, Y_train)

In [ ]:
# gruPredictTest = gru.predict(X_test)
# lassoPredictTest = lasso.predict(X_test)
# lstmPredictTest = lstm.predict(X_test)
mlpPredictTest = mlp.predict(X_test)
lightgbmPredictTest = lightgbm.predict(X_test)
# ridgePredictTest = ridge.predict(X_test)
# svmPredictTest = svm.predict(X_test)
# svmPredictTest.resize(168,1)
# svmPredictTest.resize(96,1)
# lassoPredictTest.resize(168,1)
testData = concatenate([mlpPredictTest, lightgbmPredictTest])

In [ ]:
res = model.predict(testData)
print(res)

In [ ]:
from sklearn.metrics import r2_score 
r2 = r2_score(Y_test, res) 
print(r2)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test,res)
print(mae)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(Y_test,res)
print(mape)

In [ ]:
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(Y_test,res)
print(rmse)

In [ ]:
import shap
print(testData.numpy())
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(testData.numpy())



In [ ]:
print(model.get_layer_feature_importances(0))